In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Import required modules

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Input,Model
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Input Data and Data Argumentation 

In [3]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                fill_mode='nearest')

valid_gen=ImageDataGenerator(rescale=1./255)

batch_size=50
base_dir=".\\dataset"

training_set=train_datagen.flow_from_directory(base_dir+'/train',
                                              target_size=(224,224),
                                              batch_size=batch_size,
                                              class_mode='categorical')

valid_set=valid_datagen.flow_from_directory(base_dir+'/valid',
                                              target_size=(224,224),
                                              batch_size=batch_size,
                                              class_mode='categorical')

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


In [4]:
train_num=training_set.samples
valid_num=valid_set.samples

# Transfer Learning with VGG

Download VGG-16 model without Dense layers

In [5]:
base_model=tf.keras.applications.VGG16(include_top=False,
                                      weights='imagenet',
                                      input_shape=(224,224,3))
base_model.summary()

58900480/58889256 [==============================] - 17s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_____________________________________________________________

Freeze the convolutional base created and use its as a feature extractor (by base_model.trainable=False) prevent the weights in a given layer from being updated during training

In [6]:
base_model.trainable=False

In [7]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

Build a model by channing the base_model layer and Dense layers

In [8]:
inputs=Input(shape=(224,224,3))
x=base_model(inputs,training=False)
x=GlobalAveragePooling2D()(x)
x=Dense(512,activation='relu')(x)
x=Dropout(0.2)(x)
x=Dense(512,activation='relu')(x)
x=Dropout(0.2)(x)
outputs=Dense(38,activation='softmax')(x)


vgg_model=Model(inputs,outputs)

vgg_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0     

Compile the model before training it. Since there are two classes, use a categorical cross-entropy loss & Adam optimizer

In [9]:

vgg_model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [10]:
len(vgg_model.trainable_variables)

6

In [11]:
vgg_model.evaluate(valid_set)

352/352 [==============================] - 2239s 6s/step - loss: 3.7799 - accuracy: 0.0272


[3.779949188232422, 0.027202367782592773]

# Train the model

In [13]:
weightpath='best_weights_9.hdf5'
checkpoint = ModelCheckpoint(weightpath, monitor='val_acc', verbose=1, 
                             save_best_only=True, save_weights_only=True, mode='max')

history=vgg_model.fit_generator(training_set,
                        steps_per_epoch=150,
                        epochs=15,
                        validation_data=valid_set,
                        validation_steps=100,
                        callbacks=[checkpoint])

c:\users\verma\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
150/150 [==============================] - 1549s 10s/step - loss: 2.6730 - accuracy: 0.2649 - val_loss: 1.6141 - val_accuracy: 0.5228
Epoch 2/15
150/150 [==============================] - 1647s 11s/step - loss: 1.4022 - accuracy: 0.5575 - val_loss: 1.0681 - val_accuracy: 0.6760
Epoch 3/15
150/150 [==============================] - 1605s 11s/step - loss: 1.0518 - accuracy: 0.6631 - val_loss: 0.8583 - val_accuracy: 0.7278
Epoch 4/15
150/150 [==============================] - 1531s 10s/step - loss: 0.9210 - accuracy: 0.7059 - val_loss: 0.6990 - val_accuracy: 0.7850
Epoch 5/15
150/150 [==============================] - 1540s 10s/step - loss: 0.8253 - accuracy: 0.7344 - val_loss: 0.6800 - val_accuracy: 0.7732
Epoch 6/15
150/150 [==============================] - 1681s 11s/step - loss: 0.7499 - accuracy: 0.7537 - val_loss: 0.6645 - val_accuracy: 0.7812
Epoch 7/15
150/150 [==============================] - 1754s 12s/step - loss: 0.7055 - accuracy: 0.7717 - val_loss: 0.6258 - val_ac

In [13]:
history=vgg_model.fit_generator(training_set,
                        steps_per_epoch=150,
                        epochs=15,
                        validation_data=valid_set,
                        validation_steps=100,
                        callbacks=[checkpoint])

In [14]:
history=vgg_model.fit_generator(training_set,
                        steps_per_epoch=150,
                        epochs=15,
                        validation_data=valid_set,
                        validation_steps=100,
                        callbacks=[checkpoint])

# Fine Tuning

In the feature extraction experiment, you were only training a few layers on top of an VGG-16 base model. The weights of the pre-trained network were not updated during training.

One way to increase performance even further is to train (or "fine-tune") the weights of the top layers of the pre-trained model alongside the training of the classifier you added. The training process will force the weights to be tuned from generic feature maps to features associated specifically with the dataset.

 The goal of fine-tuning is to adapt these specialized features to work with the new dataset, rather than overwrite the generic learning.

In [15]:
base_model.trainable=True #Un-Freezing the base model
vgg_model.summary()

In [16]:
vgg_model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

fine_tune_history=vgg_model.fit_generator(training_set,
                                        steps_per_epoch=150,
                                        validation_data=valid_set,
                                        epochs=10,
                                        validation_steps=100)

NameError: name 'vgg_model' is not defined